# Memory

<div style="display: flex; justify-content: flex-start; gap: 10px;">
  <img src="./assets/LC_Memory_before.png" style="width:500px; border:1px solid #ccc; border-radius:6px;">
</div>

Persisting messages, or 'agent state' between invocations of the agent.

## Setup

Load and/or check for needed environmental variables

In [ ]:
%pip install -U mlflow>=3 langchain>=1 langchain-community langgraph databricks-langchain --quiet
dbutils.library.restartPython()

In [ ]:
import mlflow
mlflow.langchain.autolog()

In [ ]:
# No Databricks, usamos spark.sql() diretamente
# Verifique as tabelas disponíveis no catálogo NASA GCN
%sql
SHOW TABLES IN nasa_gcn.silver

In [ ]:
# RuntimeContext não é necessário quando usamos spark diretamente

In [ ]:
from langchain_core.tools import tool

@tool
def execute_sql(query: str) -> str:
    """Execute a SQL command on Databricks and return results."""
    try:
        result = spark.sql(query).limit(5).toPandas()
        return result.to_markdown()
    except Exception as e:
        return f"Error: {e}"

In [ ]:
SYSTEM_PROMPT = """Você é um analista cuidadoso de Databricks SQL especializado em dados da NASA GCN.

Regras:
- Pense passo a passo.
- Quando precisar de dados, chame a ferramenta `execute_sql` com UMA query SELECT.
- Apenas leitura; sem INSERT/UPDATE/DELETE/ALTER/DROP/CREATE/REPLACE/TRUNCATE.
- Limite a 5 linhas de saída, a menos que o usuário peça mais.
- Se a ferramenta retornar 'Error:', revise o SQL e tente novamente.
- Prefira listas explícitas de colunas; evite SELECT *.
- Use o catálogo nasa_gcn e schema silver (nasa_gcn.silver.tabela).
"""

In [ ]:
from langchain.agents import create_agent
from databricks_langchain import ChatDatabricks

# Inicializar o modelo Databricks
llm = ChatDatabricks(
    endpoint="databricks-meta-llama-3-3-70b-instruct",
    temperature=0.1,
)

agent = create_agent(
    model=llm,
    tools=[execute_sql],
    system_prompt=SYSTEM_PROMPT,
)

## Repeated Queries

In [ ]:
question = "Quantos eventos existem no catálogo nasa_gcn schema silver?"
steps = []

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()
    steps.append(step)

In [ ]:
question = "Quais são os tipos de eventos mais frequentes?"

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

## Add memory

In [0]:
from langgraph.checkpoint.memory import InMemorySaver

In [ ]:
from langchain.agents import create_agent
from langchain_core.messages import SystemMessage

agent_with_memory = create_agent(
    model=llm,
    tools=[execute_sql],
    system_prompt=SYSTEM_PROMPT,
    checkpointer=InMemorySaver(),
)

In [ ]:
question = "Liste as tabelas disponíveis no catálogo nasa_gcn schema silver"
steps = []

for step in agent_with_memory.stream(
    {"messages": [{"role": "user", "content": question}]},
    {"configurable": {"thread_id": "1"}},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()
    steps.append(step)

In [ ]:
question = "Qual delas tem mais registros?"
steps = []

for step in agent_with_memory.stream(
    {"messages": [{"role": "user", "content": question}]},
    {"configurable": {"thread_id": "1"}},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()
    steps.append(step)

## Try your own queries
Now that there is memory, check the agents recall!

In [ ]:
question = "Sua pergunta aqui?"
steps = []

for step in agent_with_memory.stream(
    {"messages": [{"role": "user", "content": question}]},
    {"configurable": {"thread_id": "1"}},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()
    steps.append(step)